ADPC POUR LES DATASET REDUITS 

In [2]:
# Importer des modules
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import time

def calculate_power_sequence(power_consumption_sequence, rho_threshold, delta_threshold, d_c):
    # Initialisation des variables
    num_time_steps = len(power_consumption_sequence)
    rho_values = np.zeros(num_time_steps)
    delta_values = np.zeros(num_time_steps)

    # Calcul des distances entre toutes les paires de valeurs de consommation d'énergie
    distance_matrix = cdist(power_consumption_sequence.reshape(-1, 1), power_consumption_sequence.reshape(-1, 1), metric='euclidean')
    
    # Calcul des valeurs rho
    for i in range(1, num_time_steps):
        for j in range(i + 1, num_time_steps):
            rho_values[i] += chi(distance_matrix[i, j] - d_c)
            rho_values[j] += chi(distance_matrix[i, j] - d_c)

    # Trier les indices par valeurs rho dans l'ordre décroissant
    sorted_indices = np.argsort(-rho_values)

    # Calcul des valeurs delta
    delta_values.fill(np.max(distance_matrix) * num_time_steps)
    delta_values[sorted_indices[0]] = -1
    for i in range(1, num_time_steps):
        for j in range(i):
            if distance_matrix[sorted_indices[i], sorted_indices[j]] < delta_values[sorted_indices[i]]:
                delta_values[sorted_indices[i]] = distance_matrix[sorted_indices[i], sorted_indices[j]]

    delta_values[sorted_indices[0]] = np.max(delta_values)

    # Déterminer la séquence de puissance correspondant aux différents états de fonctionnement de l'appareil
    power_sequence = [power_consumption_sequence[i] for i in range(num_time_steps) if rho_values[i] > rho_threshold and delta_values[i] > delta_threshold]
    num_working_states = len(power_sequence)

    return sorted(power_sequence), num_working_states  # Tri des valeurs avant de les retourner


def chi(x) :
    
    return x 


# Liste des noms de fichiers CSV à lire
file_names = ['boiler_226_reduits.csv', 'computer_44_reduits.csv', 'dishwasher_53_reduits.csv', 'dryer_219_reduits.csv', 'fridge_284_reduits.csv', 'laptop_289_reduits.csv', 'tv_290_reduits.csv', 'vacuum_236_reduits.csv', 'wm_343_reduits.csv']

# Créer un DataFrame vide pour stocker les résultats
results_df = pd.DataFrame(columns=["device", "real_power", "num_states"])

# Mesurer le temps d'exécution total
total_start_time = time.time()

# Boucle à travers chaque fichier
for file_name in file_names:
    # Lire le fichier CSV dans un DataFrame
    df = pd.read_csv(file_name)
    
    # Obtenir la séquence de consommation d'énergie
    power_consumption_sequence = df['power'].values
    
    # Définissez vos paramètres spécifiques ici
    rho_threshold = 0.01 * (len(power_consumption_sequence) - max(power_consumption_sequence))
    delta_threshold = 0.1 * (max(power_consumption_sequence))
    d_c = 5
    
    # Mesurer le temps d'exécution pour chaque itération
    start_time = time.time()
    
    # Appeler la fonction calculate_power_sequence avec vos paramètres
    power_sequence, num_working_states = calculate_power_sequence(power_consumption_sequence, rho_threshold, delta_threshold, d_c)
    
    # Ajouter les résultats au DataFrame
    results_df = pd.concat([results_df, pd.DataFrame({"device": [file_name.split('_')[0]], 
                                                       "real_power": [power_sequence], 
                                                       "num_states": [num_working_states]})], ignore_index=True)

# Trier les valeurs à l'intérieur de chaque liste de résultats par ordre croissant
results_df['real_power'] = results_df['real_power'].apply(sorted)

# Mesurer le temps d'exécution total
total_execution_time = time.time() - total_start_time

# Stocker les résultats dans un nouveau fichier CSV
results_df.to_csv("dataset_appliances.csv", index=False)

# Afficher le DataFrame
print(results_df)

# Signes de démarcation
print('='*60)

# Afficher le temps d'exécution total
print(f"Total computational time: {total_execution_time:.4f} seconds")


       device                     real_power num_states
0      boiler                             []          0
1    computer             [2.0, 62.0, 280.0]          3
2  dishwasher                             []          0
3       dryer                  [0.0, 3267.0]          2
4      fridge            [1.0, 113.0, 280.0]          3
5      laptop                    [0.0, 61.0]          2
6          tv                    [1.0, 54.0]          2
7      vacuum  [207.0, 549.5, 839.0, 1099.0]          4
8          wm       [1459.5, 1961.0, 2663.0]          3
Total computational time: 14.4504 seconds
